In [1]:
import sys
sys.path.append("../../../../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pricer.evaluate import evaluate
from pricer.items import Item
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge , Lasso , LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer,StandardScaler
from tqdm.notebook import tqdm

In [3]:
# username = "ujalaarshad17"
# dataset_name = f"{username}/items_transformed_full"
# train, val, test = Item.get_from_hub(dataset_name)
# print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")
LITE_MODE = False
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"


train, val, test = Item.get_from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


In [13]:
pipeline = Pipeline(steps=[
    (
        "features",
        FeatureUnion([
            (
                "tfidf",
                TfidfVectorizer(
                    max_features=2000,
                    stop_words="english",
                    ngram_range=(1, 2),
                    sublinear_tf=True
                )
            ),
            (
                "char_len",
                FunctionTransformer(
                    lambda x: np.array([len(t) for t in x]).reshape(-1, 1),
                    validate=False
                )
            )
        ])
    ),
    (
        "normalize",
        StandardScaler(with_mean=False)
    ),
    (
        "model",
        LinearRegression()
    )
])


In [4]:
def get_dataframe(train, test):
    train_df = [
        {
            "text": len(item.summary),
            "weight": item.weight,
            "weight_unknown": 1 if item.weight == 0 else 0,
            "price": float(item.price)
        }
        for item in tqdm(train)
    ]
    
    test_df = [
        {
            "text": len(item.summary),
            "weight": item.weight,
            "weight_unknown": 1 if item.weight == 0 else 0,
            "price": float(item.price)
        }
        for item in tqdm(test)
    ]
    
    return pd.DataFrame(train_df), pd.DataFrame(test_df)


In [5]:
train_df , test_df  = get_dataframe(train , test)

  0%|          | 0/800000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
train_df.shape

(800000, 4)

In [7]:
features = ['text', 'weight', 'weight_unknown']
lr = LinearRegression()
lr.fit(train_df[features], train_df['price'])

LinearRegression()

In [8]:
def predict_lr(item):
    x = pd.DataFrame([{
        "text": len(item.summary),
        "weight": item.weight,
        "weight_unknown": 1 if item.weight == 0 else 0
    }])
    return lr.predict(x)[0]

In [9]:
predict_lr(test[0])

163.62758186604358

In [10]:
evaluate(predict_lr, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$55 $29 $87 $90 $92 $60 $1 $78 $104 $192 $556 $228 $141 $81 $50 $124 $69 $83 $50 $8 $10 $9 $61 $2 $175 $298 $345 $114 $34 $55 $100 $77 $10 $38 $24 $676 $78 $94 $50 $90 $48 $55 $83 $102 $82 $119 $100 $106 $2 $52 $106 $9 $353 $28 $92 $26 $136 $110 $37 $121 $97 $69 $43 $17 $460 $41 $83 $288 $4 $86 $109 $101 $139 $113 $99 $103 $8 $122 $122 $111 $25 $107 $98 $10 $108 $63 $60 $164 $33 $82 $109 $41 $101 $94 $114 $104 $97 $9 $151 $426 $37 $11 $135 $5 $100 $25 $107 $277 $102 $182 $90 $169 $135 $11 $45 $38 $100 $119 $98 $21 $125 $72 $76 $32 $35 $60 $109 $61 $10 $2 $78 $12 $58 $95 $58 $106 $65 $65 $1 $8 $61 $110 $54 $107 $33 $91 $93 $364 $107 $92 $118 $44 $76 $19 $111 $122 $102 $61 $138 $76 $135 $116 $83 $113 $710 $120 $269 $101 $107 $125 $110 $124 $275 $101 $52 $4 $119 $26 $48 $53 $481 $141 $139 $64 $102 $116 $19 $77 $78 $103 $94 $99 $122 $1 $30 $65 $25 $133 $112 $113 

In [11]:
def get_dataframe(train, test):
    train_df = [{"text": item.summary, "price": float(item.price)} for item in tqdm(train)]
    test_df  = [{"text": item.summary, "price": float(item.price)} for item in tqdm(test)]
    return pd.DataFrame(train_df), pd.DataFrame(test_df)

train_df, test_df = get_dataframe(train, test)


  0%|          | 0/800000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [14]:
pipeline.fit(train_df['text'], train_df['price'])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf',
                                                 TfidfVectorizer(max_features=2000,
                                                                 ngram_range=(1,
                                                                              2),
                                                                 stop_words='english',
                                                                 sublinear_tf=True)),
                                                ('char_len',
                                                 FunctionTransformer(func=<function <lambda> at 0x75bd710c2160>))])),
                ('normalize', StandardScaler(with_mean=False)),
                ('model', LinearRegression())])

In [15]:
test[0].summary

'Title: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'

In [16]:
pipeline.predict([test[0].summary])[0]

156.74209098736836

In [17]:
def predict_lr(item):
    return pipeline.predict([item.summary])[0]

In [18]:
texts = [text.summary for text in test]

In [19]:
pipeline.predict(texts)

array([156.74209099, 237.56898016,  -8.07069967, ..., 303.75693473,
       224.11170403, 147.81299834])

In [20]:
evaluate(predict_lr, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$62 $122 $63 $17 $44 $189 $29 $21 $38 $57 $539 $158 $125 $166 $28 $87 $45 $38 $50 $29 $42 $24 $22 $10 $221 $242 $121 $3 $36 $76 $53 $122 $28 $20 $99 $463 $18 $64 $169 $58 $119 $76 $62 $38 $94 $85 $24 $53 $28 $6 $24 $103 $140 $4 $72 $83 $44 $184 $11 $7 $35 $16 $0 $52 $328 $100 $26 $264 $14 $207 $20 $20 $11 $152 $43 $29 $167 $12 $24 $150 $86 $44 $23 $41 $3 $94 $57 $73 $65 $230 $22 $126 $96 $22 $14 $84 $10 $32 $109 $167 $59 $61 $20 $22 $6 $13 $53 $219 $35 $54 $22 $5 $137 $61 $18 $107 $78 $7 $42 $76 $3 $165 $114 $56 $89 $59 $25 $123 $104 $9 $76 $48 $144 $10 $84 $38 $20 $107 $80 $66 $22 $211 $77 $48 $95 $76 $42 $260 $38 $5 $85 $178 $18 $54 $62 $123 $128 $13 $5 $14 $100 $23 $62 $28 $405 $2 $71 $99 $6 $13 $26 $15 $269 $14 $36 $121 $8 $42 $21 $111 $352 $19 $42 $28 $170 $13 $32 $42 $71 $41 $104 $150 $3 $90 $56 $43 $10 $1 $68 $2 

### for selecting 5000 features we get 177, for selecting 10000 features we get 197 and for 2000 we get 187$ off

### So we performed good as compared to our mean baseline but theres a way long, now moving to Ridge and Lasso Regression

## Now moving on to Lasso and Ridge

In [21]:
lasso_pipeline = Pipeline(steps=[
    (
        "features",
        FeatureUnion([
            (
                "tfidf",
                TfidfVectorizer(
                    max_features=2000,
                    stop_words="english",
                    ngram_range=(1, 2),
                    sublinear_tf=True
                )
            ),
            (
                "char_len",
                FunctionTransformer(
                    lambda x: np.array([len(t) for t in x]).reshape(-1, 1),
                    validate=False
                )
            )
        ])
    ),
    (
        "normalize",
        StandardScaler(with_mean=False)
    ),
    (
        "model",
        Lasso()
    )
])


# Ridge pipeline
ridge_pipeline = Pipeline(steps=[
    (
        "features",
        FeatureUnion([
            (
                "tfidf",
                TfidfVectorizer(
                    max_features=2000,
                    stop_words="english",
                    ngram_range=(1, 2),
                    sublinear_tf=True
                )
            ),
            (
                "char_len",
                FunctionTransformer(
                    lambda x: np.array([len(t) for t in x]).reshape(-1, 1),
                    validate=False
                )
            )
        ])
    ),
    (
        "normalize",
        StandardScaler(with_mean=False)
    ),
    (
        "model",
        Ridge()
    )
])


In [22]:
lasso_pipeline.fit(train_df['text'], train_df['price'])
def predict_lasso(item):
    return lasso_pipeline.predict([item.summary])[0]
evaluate(predict_lasso, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$52 $82 $17 $24 $87 $157 $43 $15 $58 $93 $546 $186 $94 $157 $21 $92 $49 $39 $38 $48 $48 $61 $14 $21 $210 $276 $110 $33 $61 $50 $49 $152 $13 $54 $44 $485 $59 $83 $162 $78 $123 $96 $20 $7 $116 $70 $55 $65 $27 $18 $49 $90 $194 $1 $105 $57 $58 $190 $13 $3 $75 $46 $13 $24 $407 $106 $9 $270 $53 $166 $21 $40 $37 $148 $61 $104 $157 $12 $23 $115 $101 $66 $23 $35 $18 $43 $32 $88 $54 $203 $11 $94 $57 $24 $27 $103 $21 $15 $133 $194 $54 $46 $26 $24 $14 $33 $53 $199 $13 $75 $10 $45 $124 $80 $46 $51 $37 $4 $67 $42 $61 $133 $95 $59 $61 $54 $22 $110 $101 $1 $67 $50 $111 $35 $68 $68 $29 $61 $95 $72 $27 $138 $60 $54 $118 $75 $45 $299 $24 $26 $36 $174 $21 $52 $42 $104 $133 $46 $27 $4 $115 $8 $16 $4 $432 $14 $135 $73 $31 $36 $43 $10 $276 $29 $70 $116 $38 $11 $1 $89 $404 $55 $67 $40 $172 $65 $55 $65 $74 $68 $80 $132 $25 $68 $10 $18 $25 $20 $77 $45 

In [23]:
ridge_pipeline.fit(train_df['text'], train_df['price'])
def predict_ridge(item):
    return ridge_pipeline.predict([item.summary])[0]
evaluate(predict_ridge, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$62 $122 $63 $17 $44 $189 $29 $21 $38 $57 $539 $158 $125 $166 $28 $87 $45 $38 $50 $29 $42 $24 $22 $9 $221 $242 $121 $3 $36 $76 $53 $122 $28 $20 $99 $463 $18 $64 $169 $58 $119 $76 $62 $38 $94 $85 $24 $53 $28 $6 $24 $102 $140 $4 $72 $83 $44 $185 $11 $7 $35 $16 $0 $52 $329 $100 $26 $264 $14 $207 $20 $20 $11 $152 $43 $29 $167 $12 $24 $150 $86 $44 $23 $41 $3 $94 $57 $73 $65 $229 $22 $126 $96 $22 $14 $84 $10 $32 $109 $167 $59 $61 $20 $22 $6 $13 $53 $219 $35 $54 $22 $5 $137 $61 $18 $107 $78 $7 $42 $76 $3 $165 $114 $56 $89 $59 $25 $123 $105 $9 $76 $48 $144 $10 $84 $38 $20 $107 $80 $66 $22 $211 $77 $48 $95 $76 $42 $260 $38 $4 $86 $178 $18 $54 $62 $123 $128 $13 $5 $14 $100 $23 $62 $28 $405 $2 $71 $99 $6 $13 $26 $15 $269 $14 $36 $121 $7 $42 $21 $111 $352 $19 $42 $28 $170 $13 $32 $42 $71 $41 $104 $150 $3 $90 $56 $43 $10 $1 $68 $2 

# Final Remarks: LR and Ridge Regression almost performed the same so we got our best value till now and we are off by 75$. Los gehts zum SVM